In [11]:
# 이미지 변환

import cv2
import numpy as np

# 이미지를 컬러로 로드하여 넘파이 배열 형태로 img에 대입
img = cv2.imread('image.jpg',cv2.IMREAD_COLOR)

# 이미지가 저장된 넘파이 배열의 너비와 높이를 구한다.
height, width = img.shape[:2]
print("height", height,"width", width)

# 이동 변환 행렬을 생성, x방향 100픽셀, y방향으로 200픽셀
M = np.array([[1,0,100],[0,1,200],[0,0,1]],dtype=float)

# 결과 이미지를 저장할 넘파이 배열을 입력이미지와 같은 크기로 생성
dst = np.zeros(img.shape, dtype = np.uint8)

# 이미지 전체 픽셀을 스캔
for y in range(height-1):
    for x in range(width-1):
        # 현재 픽셀 p의 좌표에 이동 변환 행렬 M을 곱하여 이동시킨 좌표
        # p_를 구합니다.
        p = np.array([x,y,1])
        p_= np.dot(M,p)

        # 이동한 위치인 p_에서 좌표 x_, y_를 정수형태로 가져옵니다.
        x_, y_ = p_[:2]
        x_ = int(x_)
        y_ = int(y_)

        # 입력 이미지 크기 내로 픽셀이 이동한 경우에만
        # 결과 이미지를 저장할 넘파이 배열로 픽셀을 복사
        if x_ > 0 and x_ < width and y_ > 0 and y_ < height:
            dst[y_,x_] = img[y,x]

# 입력 이미지와 결과 이미지를 수평으로 연결하여 하나의 이미지를 만든다
result = cv2.hconcat([img,dst])

# title 추가, 윈도우에 이미지를 result를 보여줌
cv2.imshow("result",result)
cv2.waitKey(0)

height 650 width 866


13

In [12]:
import cv2
import numpy as np


img = cv2.imread('image.jpg', cv2.IMREAD_COLOR)
height,width = img.shape[:2]

# 45도를 라디안으로 변환하여 코싸인값과 싸인값을 구합니다.
angle = 45
radian = angle*np.pi/180
c = np.cos(radian)
s = np.sin(radian)

# 이미지 중심 좌표를 구합니다. 
center_x = width/2
center_y = height/2

# 회전변환행렬을 구성합니다.
rotation_matrix = np.array([[c, s, (1-c)*center_x-s*center_y], [-s, c, s*center_x+(1-c)*center_y], [0, 0, 1]])


dst = np.zeros(img.shape, dtype=np.uint8)

for y in range(height-1):
	for x in range(width-1):

		# backward mapping
		# 결과 이미지의 픽셀 new_p로 이동하는 입력 이미지의 
		# 픽셀 old_p의 위치를 계산합니다.
		new_p = np.array([x, y, 1])
		inv_rotation_matrix = np.linalg.inv(rotation_matrix)
		old_p = np.dot(inv_rotation_matrix, new_p)

		# new_p 위치에 계산하여 얻은 old_p 픽셀의 값을 대입합니다.
		x_,y_ = old_p[:2]
		x_ = int(x_)
		y_ = int(y_)

		if x_ > 0 and x_ < width and y_ > 0 and y_ < height:
			dst[y, x] = img[y_, x_]

			
result = cv2.hconcat([img, dst]) 
cv2.imshow("result", result)
cv2.waitKey(0)


13

In [13]:
import cv2
import numpy as np


img = cv2.imread('image.jpg', cv2.IMREAD_COLOR)
height,width = img.shape[:2]

# 45도를 라디안으로 변환하여 코싸인값과 싸인값을 구합니다.
angle = 45
radian = angle*np.pi/180
c = np.cos(radian)
s = np.sin(radian)

# 이미지 중심 좌표를 구합니다. 
center_x = width/2
center_y = height/2

# 좌표 (center_x, center_y) 중심으로 angle 만큼 회전하는 회전 변환 행렬을
# 생성합니다.
rotation_matrix = cv2.getRotationMatrix2D((center_x,center_y),angle,1)

# 이미지에 회전 변환 행렬을 적용
dst = cv2.warpAffine(img,rotation_matrix,(width,height))

result = cv2.hconcat([img,dst])
cv2.imshow("result",result)
cv2.waitKey(0)

27

In [14]:
import cv2
import numpy as np


img = cv2.imread('image.jpg', cv2.IMREAD_COLOR)
height,width = img.shape[:2]


scale_factor = 0.5
scaling_matrix = np.array([[scale_factor, 0, 0], [0, scale_factor, 0], [0, 0, 1]])
translation_matrix = np.array([[1, 0, width/4], [0, 1, height/4], [0, 0, 1]])
angle = 45
radian = angle*np.pi/180
c = np.cos(radian)
s = np.sin(radian)
center_x = width / 2
center_y = height / 2
rotation_matrix = np.array([[c, s, (1-c)*center_x-s*center_y], [-s, c, s*center_x+(1-c)*center_y], [0, 0, 1]])

# 정해진 순서대로 변환 행렬을 곱하여 하나의 행렬을 생성합니다. 
T = np.eye(3)
T = np.dot(scaling_matrix, T)
T = np.dot(translation_matrix, T)
T = np.dot(rotation_matrix, T)

dst = np.zeros((height, width, img.shape[2]) , dtype=np.uint8)

for y in range(height):
	for x in range(width):

             # 미리 구해놓은 변환행렬을 행렬곱 한번으로 적용합니다. 
             # 여기에서도 backward mapping을 사용합니다. 
		new_p = np.array([x, y, 1])
		inv_scaling_matrix = np.linalg.inv(T)
		old_p = np.dot(inv_scaling_matrix, new_p)

		x_,y_ = old_p[:2]
		x_ = int(x_)
		y_ = int(y_)

		if x_ > 0 and x_ < width and y_ > 0 and y_ < height:
			dst.itemset((y, x, 0), img.item(y_, x_, 0))
			dst.itemset((y, x, 1), img.item(y_, x_, 1))
			dst.itemset((y, x, 2), img.item(y_, x_, 2))
			 
cv2.imshow("result", dst)
cv2.waitKey(0)


27